In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import eig

In [2]:
data = [
    ['AAA', '90.788%', '8.291%', '0.716%', '0.102%', '0.102%', '0', '0', '0'],
    ['AA', '0.103%', '91.219%', '7.851%', '0.620%', '0.103%', '0.103%', '0', '0'],
    ['A', '0.924%', '2.361%', '90.041%', '5.441%', '0.719%', '0.308%', '0.103%', '0.103%'],
    ['BBB', '0', '0.318%', '5.938%', '86.947%', '5.302%', '1.1166%', '0.117%', '0.212%'],
    ['BB', '0', '0.110%', '0.659%', '7.692%', '80.549%', '8.791%', '0.989%', '1.209%'],
    ['B', '0', '0.114%', '0.227%', '0.454%', '6.470%', '82.747%', '4.086%', '5.902%'],
    ['CCC', '0', '0', '0.456%', '1.251%', '2.275%', '12.856%', '60.637%', '22.526%'],
    ['Default', '0', '0', '0', '0', '0', '0', '0', '100%']
]

phi = pd.DataFrame(data, columns=["Index",'AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default'])
phi.set_index("Index", inplace=True)

# Remove '%' and convert to float
phi = phi.replace(r'%', '', regex=True).astype(float)/100

### (a) Two Step Transition Matrix

In [4]:
two_step = (phi.dot(phi)*100)
two_step

,AAA,AA,A,BBB,BB,B,CCC,Default
Index,,,,,,,,
AAA,82.439765,15.107542,1.952391,0.279497,0.193860,0.020851,0.001866,0.002187
AA,0.260010,83.405164,14.269188,1.540298,0.273011,0.219344,0.014039,0.016725
A,1.673292,4.374602,81.594788,9.703490,1.540669,0.671822,0.181260,0.257350
BBB,0.055195,0.713869,10.572522,76.337229,8.998565,2.394560,0.276850,0.558802
BB,0.006202,0.238986,1.614041,12.972614,65.885373,14.570529,1.765208,2.941450
B,0.002215,0.212241,0.489407,1.332257,10.684028,69.570620,5.923424,11.785559
CCC,0.004213,0.031903,0.805551,2.104446,4.113371,18.648815,37.318187,36.974478
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000


In [5]:
print(f'The probability that a AAA bond gets rated as default within two years is: {two_step.T['AAA']['Default']}%')

The probability that a AAA bond gets rated as default within two years is: 0.0021869%


### (b) Left Eigenvectors and Eigenvalues

In [7]:
# Compute left eigenvectors and eigenvalues by transposing the matrix
eigenvalues_left, eigenvectors_left = np.linalg.eig(phi.T.to_numpy())

In [8]:
eigenvalues_left

array([1.        +0.j        , 0.58444087+0.j        ,
       0.72999885+0.j        , 0.98781601+0.j        ,
       0.8244802 +0.j        , 0.93102012+0.j        ,
       0.88576197+0.01505463j, 0.88576197-0.01505463j])

In [9]:
eigenvectors = pd.DataFrame(eigenvectors_left)
eigenvectors

,0,1,2,3,4,5,6,7
0,0.0+0.0j,0.000049+0.000000j,-0.006952+0.000000j,-0.035749+0.000000j,0.063117+0.000000j,-0.067658+0.000000j,-0.146145-0.112194j,-0.146145+0.112194j
1,0.0+0.0j,0.001909+0.000000j,-0.008211+0.000000j,-0.143127+0.000000j,0.077336+0.000000j,-0.354986+0.000000j,-0.180173+0.273146j,-0.180173-0.273146j
2,0.0+0.0j,-0.001931+0.000000j,0.134743+0.000000j,-0.293314+0.000000j,-0.578313+0.000000j,-0.129867+0.000000j,0.552712+0.000000j,0.552712-0.000000j
3,0.0+0.0j,-0.043713+0.000000j,-0.429839+0.000000j,-0.220958+0.000000j,0.644645+0.000000j,0.348954+0.000000j,0.172229-0.206578j,0.172229+0.206578j
4,0.0+0.0j,0.058065+0.000000j,0.729701+0.000000j,-0.112470+0.000000j,0.062099+0.000000j,0.365195+0.000000j,-0.256831-0.035768j,-0.256831+0.035768j
5,0.0+0.0j,-0.446654+0.000000j,-0.477854+0.000000j,-0.095304+0.000000j,-0.421484+0.000000j,0.420901+0.000000j,-0.480951+0.058850j,-0.480951-0.058850j
6,0.0+0.0j,0.808474+0.000000j,-0.102504+0.000000j,-0.014595+0.000000j,-0.075417+0.000000j,0.064945+0.000000j,-0.076100+0.010576j,-0.076100-0.010576j
7,1.0+0.0j,-0.376271+0.000000j,0.160160+0.000000j,0.906331+0.000000j,0.229846+0.000000j,-0.645004+0.000000j,0.416063+0.011190j,0.416063-0.011190j


Thus, the eigenvector that corresponds to the eigenvalues $\lambda = 1$ is the first column of the above matrix, i.e.:

In [11]:
v_T = [str(int(v.real)*100) + '%' for v in eigenvectors_left.T[0]]
v_T

['0%', '0%', '0%', '0%', '0%', '0%', '0%', '100%']

Confirmation $v^T\times \Phi = v^T$

In [13]:
v_x_Phi = [str(int(v.real)*100) + '%' for v in eigenvectors[0].values.dot(phi.values)]
v_x_Phi

['0%', '0%', '0%', '0%', '0%', '0%', '0%', '100%']

In [14]:
assert v_T == v_x_Phi, "You didn't get the right left eigenvector for the eigenvalue 1"